In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
yellow_j=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [3]:
yellow_f=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [4]:
#number of columns present in the jan 2023 yellow cab data set
Columns=len(yellow_j.columns)
print ("No of Columns yellow cab January 2023 = ", Columns)

No of Columns yellow cab January 2023 =  19


In [5]:
#convert both duration columns to date time

yellow_j['tpep_pickup_datetime'] = pd.to_datetime(yellow_j['tpep_pickup_datetime'])
yellow_j['tpep_dropoff_datetime'] = pd.to_datetime(yellow_j['tpep_dropoff_datetime'])

#solve for the duration in minutes and return to the df

yellow_j['duration']=(yellow_j['tpep_dropoff_datetime'] - yellow_j['tpep_pickup_datetime']).dt.total_seconds()/60

print ("Standard Deviation of the Duration of Yellow Cabs January 2023 = ", np.std(yellow_j['duration']))

Standard Deviation of the Duration of Yellow Cabs January 2023 =  42.5943442974141


In [6]:
#drop rows
yellow_jan = yellow_j[(yellow_j['duration'] >= 1) & (yellow_j['duration'] <= 60)]

rows_Yjan=yellow_jan.shape[0]

print("Yellow cabs observations Jan 2023, duration >= 1 or <=60 =",rows_Yjan)

rows_Yj=yellow_j.shape[0]

print("Yellow cabs observations Jan 2023",rows_Yj)

prop_n=rows_Yjan/rows_Yj

print("Propotion of dropped outliers = {:.3%}". format(prop_n))

Yellow cabs observations Jan 2023, duration >= 1 or <=60 = 3009173
Yellow cabs observations Jan 2023 3066766
Propotion of dropped outliers = 98.122%


In [7]:
#Feature Selection
categorical=['PULocationID','DOLocationID']

yellow_jan[categorical] = yellow_jan[categorical].astype(str)

train_dicts = yellow_jan[categorical].to_dict(orient='records')

train_dicts[1:5]

/tmp/ipykernel_169333/2926570742.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yellow_jan[categorical] = yellow_jan[categorical].astype(str)


[{'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [8]:
dv = DictVectorizer(sparse=True)

feature_matrix = dv.fit_transform(train_dicts)



X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = yellow_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mse=mean_squared_error(y_train, y_pred, squared=False)

print(f"4, Dimension of feature_matrix: {feature_matrix.shape} \n")
print("Mean Square Error =",mse)

4, Dimension of feature_matrix: (3009173, 515) 

Ean Square Error = 7.649261027919939


In [9]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    
    #convert both duration columns to date time

    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    #solve for the duration in minutes and return to the df

    df['duration']=(df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])

    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    #drop rows
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    #Feature Selection
    categorical=['PULocationID','DOLocationID']
    numerical=['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [10]:
df_val=read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
df_train=yellow_jan

In [11]:
categorical = ['PULocationID','DOLocationID'] 

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.811832638273232